# Imports

In [2]:
import json
import tensorflow as tf
import tensorflow.keras.backend as K
import numpy as np 
import pandas as pd
import seaborn as sns
import os
import gc
import rasterio as rio
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import  cm
# import cv2
from matplotlib import animation
from IPython.display import HTML
from tqdm import tqdm
from sklearn.model_selection import train_test_split

2024-11-10 01:18:26.090034: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-10 01:18:26.110148: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-10 01:18:26.130623: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-10 01:18:26.136846: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-10 01:18:26.154236: I tensorflow/core/platform/cpu_feature_guar

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/ea301b/.local/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/ea301b/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/ea301b/.local/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/ea301b/.local/

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/home/ea301b/.local/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/ea301b/.local/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/home/ea301b/.local/lib/python3.11/site-packages/ipykernel/kernelapp.py", line 739, in start
    self.io_loop.start()
  File "/home/ea301b/.local/

AttributeError: _ARRAY_API not found

SystemError: initialization of _pywrap_checkpoint_reader raised unreported exception

# Config

In [ ]:
class CFG:
    seed = 10
    img_size = (256,256)
    BATCH_SIZE = 3
    Autotune = tf.data.AUTOTUNE
    validation_size = 0.2
    class_dict= {0:'No Flooding', 
                 1: 'Flooding'}
    
    test_run = False 
    


# Config

In [ ]:
class CFG:
    seed = 10
    img_size = (256,256)
    BATCH_SIZE = 3
    Autotune = tf.data.AUTOTUNE
    validation_size = 0.2
    class_dict= {0:'No Flooding', 
                 1: 'Flooding'}
    
    test_run = False 
    


# Input data

    Read more about the dataset here : https://clmrmb.github.io/SEN12-FLOOD/

In [ ]:
s1_labels = './archive/sen12flood/sen12floods_s1_labels/sen12floods_s1_labels/'
s1_tiles = './archive/sen12flood/sen12floods_s1_source/sen12floods_s1_source/'

s2_tiles = './archive/sen12flood/sen12floods_s2_source/sen12floods_s2_source/'
s2_labels = './archive/sen12flood/sen12floods_s2_labels/sen12floods_s2_labels/'


s1_check = 0
for file in os.listdir(s1_labels):
    if os.path.exists(s1_tiles + '/' + file.replace('labels','source')):
        s1_check += 1
        
         
assert s1_check == len(os.listdir(s1_tiles)), 'You my friend , are definintely a idiot!'
    
s2_check = 0
for file in os.listdir(s2_labels):
    if os.path.exists(s2_tiles + '/' + file.replace('labels','source')):
        s2_check += 1
        
        
assert s2_check == len(os.listdir(s2_tiles)), 'You my friend , are definintely  the idiot!'


s1_check,s2_check 

(3332, 2237)

# Make a dataset of paths and labels

# Helper Functions

In [ ]:
def load_json(path):
    '''loads a json file'''
    with open(path,'r') as file:
        js = json.load(file)
        
    return js

# collectionss1 = load_json('../input/sen12flood/sen12flood/sen12floods_s1_source/sen12floods_s1_source/collection.json')
# collections2= load_json('../input/sen12flood/sen12flood/sen12floods_s2_source/sen12floods_s2_source/collection.json')
# collections2

In [ ]:
def process_label_json(label_json):
    '''process a single label json'''
    info_dict = {}
    
    info_dict['geometry'] = label_json['geometry']['coordinates']
    info_dict['label'] = label_json['properties']['FLOODING']
    info_dict['date'] = label_json['properties']['date']
    info_dict['tile_number'] = label_json['properties']['tile']
#     info_dict['full_data_coverage']= label_json['properties']['FULL-DATA-COVERAGE']
    
    return info_dict


def process_label_stac(stac_json):
    return stac_json['id']
    
    


def image_path_from_label_dir(image_parent_dir:str,
                              label_file :str)->str:
    
    return image_parent_dir + '/' + label_file.replace('labels','source')
    
    

def process_json(label_path,image_directory):
    '''get the data for a single example
     Inputs 
     label_path : path to the label folder 
     image_directory: path to the corresponding image directory'''
    
    

    #get image directory for that label
    folder_id = label_path.rsplit('/',1)[1]
    image_dir_path = image_path_from_label_dir(image_directory,folder_id)

    if not os.path.exists(image_dir_path):
        return {'File_not_found':image_dir_path}
    
    
    for file in os.listdir(label_path):
        #if image dir exists 
        if file.startswith('labels'):
            label_json = load_json(os.path.join(label_path,file))
        else:
            stac_json = load_json(os.path.join(label_path,file))


    #get data 
    info_dict = process_label_json(label_json)

    #get id 
    info_dict['id'] = process_label_stac(stac_json)
    
    #location id 
    info_dict['location_id'] = info_dict['id'].split('_')[3]
    
    
    info_dict['image_dir'] = image_dir_path
    
    
    return info_dict


In [ ]:
def get_dataframe(label_directory,image_directory):
    '''get dataframe from the nested label directory'''
    records = []
    
        
    for folder in os.listdir(label_directory):
        if folder.startswith('sen12'):
#             print(folder,label_directory)
            folder_path = label_directory + '/' + folder
            
            
            #get data for a single example
            feature = process_json(label_path=folder_path,
                                   image_directory=image_directory)
            
            
            records.append(feature)
            
            
    return pd.DataFrame.from_records(data = records)



def type_cast_dataset(dataset):
    '''typecasting columns in dataset'''
    dataset['label'] = dataset['label'].astype(int)
    
    dataset['date'] = pd.to_datetime(dataset['date'])
    dataset['tile_number'] = dataset['tile_number'].astype('int8')
    
    
    return dataset
    

In [ ]:
%%time
s1_data = type_cast_dataset(
                            get_dataframe(
                                label_directory=s1_labels,
                                image_directory=s1_tiles
                                        )
                            )


s2_data = type_cast_dataset(
                            get_dataframe(label_directory=s2_labels,
                                          image_directory=s2_tiles)
                            )

print(f'Number of unique locations in Sentinel1 (SAR) data : {s1_data.location_id.nunique()}')
print(f'Number of unique locations in Sentinel2 (optical) data : {s2_data.location_id.nunique()}')

s1_data.shape,s2_data.shape

Number of unique locations in Sentinel1 (SAR) data : 335
Number of unique locations in Sentinel2 (optical) data : 335
CPU times: user 496 ms, sys: 160 ms, total: 656 ms
Wall time: 656 ms


((3331, 7), (2236, 7))

In [ ]:
# saving datasets
s1_data.to_csv('s1_data.csv',index=False)
s2_data.to_csv('s2_data.csv',index=False)

In [ ]:
def load_raster(filepath):
    '''load a single band raster'''
    with rio.open(filepath) as file: 
        raster = file.read().squeeze(axis=0)
        
    return raster



**Loading multiple raster bands as single raster**

In [ ]:
def load_s1_tiffs(folder,
                  scaling_values=[50.,100.]):
    images = []
    i = 0
    for im in sorted(os.listdir(folder)):
         
        if im.rsplit('.',maxsplit=1)[1] == 'tif':
            
            path = folder + '/' + im
            band = load_raster(path)
            band = band / scaling_values[i]
            
            band = cv2.resize(band,
                              CFG.img_size)
            
            images.append(band)
            i+=1 
                    
    return np.dstack(images)


def load_s2_tiffs(folder,
                  scaling_value=10000.):
    images = []
    for im in sorted(os.listdir(folder)):
        if im.rsplit('.',maxsplit=1)[1] == 'tif':    
            path = folder + '/' + im
            band = load_raster(path)
            band = band/ scaling_value
            
            band = cv2.resize(band,CFG.img_size)
            images.append(band)   

    return np.dstack(images)
                    
def load_rgb_tiffs(folder,
                  scaling_value=10000.):
    '''load R,G and B bands'''
    
    images = []
    for im in sorted(os.listdir(folder)):
        name,file_format = im.rsplit('.',maxsplit=1)
        if ((file_format== 'tif') and (name in ['B02','B03','B04'])):    
            path = folder + '/' + im
            band = load_raster(path)
            band = band/ scaling_value
            
            band = cv2.resize(band,CFG.img_size)
            images.append(band)   

    return np.dstack(images)[:,:,::-1]


    
def tf_load_s1(path):    
    path = path.numpy().decode('utf-8')
    return load_s1_tiffs(path)
    
    

def tf_load_s2(path):    
    path = path.numpy().decode('utf-8')
    return load_s2_tiffs(path)


def tf_load_rgb(path):    
    path = path.numpy().decode('utf-8')
    return load_rgb_tiffs(path)
    
def process_image_s1(filename):
    '''function for preprocessing in tensorflow data'''
    
    image = tf.py_function(tf_load_s1, [filename], tf.float32)
    image.set_shape([None, None, 2])  # Explicitly set the shape (assuming grayscale images)
    return image



def process_image_s2(filename):
    '''function for preprocessing in tensorflow data'''
    
    return tf.py_function(tf_load_s2, 
                          [filename], 
                          tf.float32)



def process_image_rgb(filename):
    '''function for preprocessing in tensorflow data'''
    
    image = tf.py_function(tf_load_rgb, 
                          [filename], 
                          tf.float32)
    image.set_shape([None,None,3])
    return image
    

In [ ]:
def count_rasters_in_folder(path):
    count = 0 
    
    for file in os.listdir(path):
        if file.rsplit('.',1)[1] == 'tif':
            count +=1 
            
    return count 
    
    
s2_data['raster_count'] = s2_data.image_dir.apply(lambda x : count_rasters_in_folder(x))

#value counts 
s2_data['raster_count'].value_counts()


s2_data=s2_data[s2_data['raster_count']==12] # take only valid rasters
# s2_data[s2_data['raster_count']==0]['location_id'].value_counts()

# Making a dataset

    First lets split the dataset into training and validation set. We will stratify based on location id to ensure that locations are well represented in traininng and validation set

In [ ]:
#isolating single loaction ids (as they will be a problem for stratification)

# single example locations 
single_index = s2_data['location_id'].value_counts()[s2_data['location_id'].value_counts()==1].index

single_index_df = s2_data[s2_data['location_id'].isin(single_index)].reset_index(drop=True)
s2_data0 = s2_data[~(s2_data['location_id'].isin(single_index))].reset_index(drop=True)

s2_data0.shape,single_index_df.shape

((2126, 8), (12, 8))

**Split dataset into train and validation splits**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split  # Assuming you have scikit-learn installed

# Train-test split with stratification
s1_data_tr, s1_data_val = train_test_split(s1_data,
                                          test_size=CFG.validation_size,
                                          random_state=CFG.seed,
                                          stratify=s1_data.location_id)

s2_data_tr, s2_data_val = train_test_split(s2_data0,
                                          test_size=CFG.validation_size,
                                          random_state=CFG.seed,
                                          stratify=s2_data0.location_id)

# Concatenate s2_data_tr and single_index_df while resetting index
s2_data_tr = pd.concat([s2_data_tr, single_index_df], ignore_index=True)

# No need for del or gc.collect() here (automatic memory management)

In [ ]:
s1_data_tr.label.value_counts(1),s1_data_val.label.value_counts(1)

(label
 0    0.693318
 1    0.306682
 Name: proportion, dtype: float64,
 label
 0    0.67916
 1    0.32084
 Name: proportion, dtype: float64)

In [ ]:
s2_data_tr.label.value_counts(1),s2_data_val.label.value_counts(1)

(label
 0    0.768107
 1    0.231893
 Name: proportion, dtype: float64,
 label
 0    0.746479
 1    0.253521
 Name: proportion, dtype: float64)

**Function for image augmentations**

    Although the Augmentations are simple, we cannot use them on SAR images , as even simple operations like flipping can change the meaning of the image

In [ ]:
def augment_image_multispectral(image):
    '''perform simple image augmentations'''
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_crop(image, size=(*CFG.img_size,12))
    
    rot = tf.random.normal((1,),mean = 0.35, stddev=0.15)
    
    if rot > 0.5:
        image = tf.image.rot90(image)

    return image 

def augment_image(image):
    '''perform simple image augmentations'''
    image = tf.image.random_flip_left_right(image)
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_crop(image, size=(*CFG.img_size,3))
    
    rot = tf.random.normal((1,),mean = 0.35, stddev=0.15)
    
    if rot > 0.5:
        image = tf.image.rot90(image)

    return image 

In [ ]:
def get_tf_dataset(image_paths,
                   labels=None, # put none for test data set
                   image_processing_fn=None,
                   augment_fn = None
                  ):
    
    '''returns a tf dataset object
    Inputs: 
    image_paths : paths to images
    labels: labels of each image
    image_processing_fn:  function to load and preprocess images 
    augment_fn : function to augment images '''
    
    #seperate datasets
    if labels is not None:
        labels_dataset = tf.data.Dataset.from_tensor_slices(labels)
    
    
    
    image_dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    #load images 
    image_dataset = image_dataset.map(image_processing_fn,
                                      num_parallel_calls=tf.data.AUTOTUNE)
     
    if augment_fn is not None:
        
        image_dataset = image_dataset.map(augment_fn,
                                          num_parallel_calls=tf.data.AUTOTUNE)
     
    
    if labels is not None:
        return tf.data.Dataset.zip((image_dataset,labels_dataset))
    
    
    return image_dataset



def optimize_pipeline(tf_dataset,
                      batch_size = CFG.BATCH_SIZE,
                      Autotune_fn = CFG.Autotune,
                      cache= False,
                      batch = True):
    
    
    
    # prefetch(load the data with cpu,while gpu is training) the data in memory 
    tf_dataset = tf_dataset.prefetch(buffer_size=Autotune_fn)  
    if cache:
        tf_dataset = tf_dataset.cache()                        # store data in RAM  
        
    tf_dataset =  tf_dataset.shuffle(buffer_size=50)         # shuffle 
    
    if batch:
        tf_dataset = tf_dataset.batch(batch_size)              #split the data in batches  
    
    return tf_dataset

**Making dataset pipelines with TF data**

In [ ]:
# Sentinel 1 dataset (not using augmentation here)

S1_dataset_tr = optimize_pipeline(tf_dataset=get_tf_dataset(image_paths = s1_data_tr.image_dir.values,
                                               labels = tf.one_hot(s1_data_tr.label,depth=2),
                                               image_processing_fn = process_image_s1),
                                  
                                  batch_size = 3 * CFG.BATCH_SIZE)


S1_dataset_val = optimize_pipeline(tf_dataset=get_tf_dataset(image_paths = s1_data_val.image_dir.values,
                                                labels = tf.one_hot(s1_data_val.label,depth=2),
                                                image_processing_fn = process_image_s1),
                                   batch_size = 3* CFG.BATCH_SIZE)

2024-11-10 01:17:27.134280: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
#sentinel 2 dataset 
S2_dataset_tr = optimize_pipeline(get_tf_dataset(image_paths = s2_data_tr.image_dir.values,
                                                   labels = s2_data_tr.label,
                                                   image_processing_fn = process_image_s2,
                                                   augment_fn = augment_image_multispectral)
                                 )


S2_dataset_val = optimize_pipeline(get_tf_dataset(image_paths = s2_data_val.image_dir.values,
                                                   labels = s2_data_val.label,
                                                   image_processing_fn = process_image_s2,
                                                   augment_fn = augment_image_multispectral)
                                  )




In [ ]:
RGB_dataset_tr = optimize_pipeline(get_tf_dataset(image_paths = s2_data_tr.image_dir.values,
                                                   labels = s2_data_tr.label,
                                                   image_processing_fn = process_image_rgb,
                                                   augment_fn = augment_image),
                                   batch_size = 3* CFG.BATCH_SIZE
                                 )


RGB_dataset_val = optimize_pipeline(get_tf_dataset(image_paths = s2_data_val.image_dir.values,
                                                   labels = s2_data_val.label,
                                                   image_processing_fn = process_image_rgb,
                                                   augment_fn = augment_image),
                                    batch_size = 3* CFG.BATCH_SIZE
                                  )



**Checking to see if the Pipelines work as expected**

In [ ]:
for x,y in S1_dataset_val.take(1): # take one batch for checking 
    print(f'shape of SAR dataset input(val) {x.shape}')

2024-11-10 01:17:28.121233: W tensorflow/core/framework/op_kernel.cc:1828] UNKNOWN: NameError: name 'rio' is not defined
Traceback (most recent call last):

  File "/home/ea301b/anaconda3/envs/Pennylane/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/ea301b/anaconda3/envs/Pennylane/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/ea301b/anaconda3/envs/Pennylane/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 153, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  File "/home/ea301b/anaconda3/envs/Pennylane/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/tmp/ipykernel_40

UnknownError: {{function_node __wrapped__IteratorGetNext_output_types_2_device_/job:localhost/replica:0/task:0/device:CPU:0}} Error in user-defined function passed to ParallelMapDatasetV2:9 transformation with iterator: Iterator::Root::Prefetch::FiniteTake::BatchV2::Shuffle::Prefetch::Zip[0]::ParallelMapV2: NameError: name 'rio' is not defined
Traceback (most recent call last):

  File "/home/ea301b/anaconda3/envs/Pennylane/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/ea301b/anaconda3/envs/Pennylane/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/home/ea301b/anaconda3/envs/Pennylane/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 153, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  Fil

In [ ]:
for x,y in S1_dataset_tr.take(1): # take one batch for checking 
    print(f'shape of SAR dataset input {x.shape}')

shape of SAR dataset input (9, 256, 256, 2)


2024-11-10 01:16:29.759624: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [ ]:
for x,y in S2_dataset_tr.take(1): # take one batch for checking 
    print(f'shape of MultiSpectral dataset input {x.shape}')

shape of MultiSpectral dataset input (3, 256, 256, 12)


In [ ]:
for x,y in RGB_dataset_tr.take(1): # take one batch for checking 
    print(f'shape of MultiSpectral dataset input {x.shape}')

shape of MultiSpectral dataset input (9, 256, 256, 3)


2024-11-10 01:16:42.582838: I tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
